In [13]:
# Importing the required libraries. Refer to the .yml file for the full conda environment.
import xarray as xr

# Dataset Preparation and Interpolation:

The datasets for this project need to have the same number of grid points and must cover the same time range. Otherwise, comparisons between the datasets will be difficult, if not impossible. So let's load our datasets and begin!

In [14]:
# Set the file paths for both of my datasets: 
# Note: these file paths are for my local computer. Please change them if necessary. 
f_path_GPCP = 'precip.mon.mean.nc'
f_path_ERA5 = 'ERA5 Monthly Averaged on Single Levels [Total Precipitation].nc'

In [15]:
# Load the GPCP Dataset using xarray:
GPCP = xr.load_dataset(f_path_GPCP)
GPCP

<xarray.Dataset> Size: 23MB
Dimensions:    (time: 559, nv: 2, lat: 72, lon: 144)
Coordinates:
  * time       (time) datetime64[ns] 4kB 1979-01-01 1979-02-01 ... 2025-07-01
  * lat        (lat) float32 288B -88.75 -86.25 -83.75 ... 83.75 86.25 88.75
  * lon        (lon) float32 576B 1.25 3.75 6.25 8.75 ... 353.8 356.2 358.8
Dimensions without coordinates: nv
Data variables:
    time_bnds  (time, nv) datetime64[ns] 9kB 1979-01-01 ... 2025-08-01
    lat_bnds   (lat, nv) float32 576B -90.0 -87.5 -87.5 -85.0 ... 87.5 87.5 90.0
    lon_bnds   (lon, nv) float32 1kB 0.0 2.5 2.5 5.0 ... 355.0 357.5 357.5 360.0
    precip     (time, lat, lon) float32 23MB 0.0 0.0 0.0 ... 1.98 1.976 1.978
Attributes: (12/18)
    Conventions:           CF-1.0
    curator:               Dr. Jian-Jian Wang\nESSIC, University of Maryland ...
    citation:              Adler, R.F., G.J. Huffman, A. Chang, R. Ferraro, P...
    title:                 GPCP Version 2.3 Combined Precipitation Dataset (F...
    platform:              NOAA POES (Polar Orbiting Environmental Satellites)
    source_obs:            CDR RSS SSMI/SSMIS Tbs over ocean \nCDR SSMI/SSMIS...
    ...                    ...
    source:                https://www.ncei.noaa.gov/data/global-precipitatio...
    source_documentation:  https://www.ncdc.noaa.gov/cdr/atmospheric/precipit...
    NCO:                   4.6.9
    history:               Generated at NOAA/ESRL PSD Sep 9 2016 based on dat...
    References:            http://www.psl.noaa.gov/data/gridded/data.gpcp.html
    data_comment:          Interim data covers 2025/06 through latest.

In [16]:
# Load the ERA5 Dataset using xarray:
# Note: this one may take some time as the file is large.
ERA5 = xr.load_dataset(f_path_ERA5)
ERA5

<xarray.Dataset> Size: 4GB
Dimensions:     (valid_time: 1028, latitude: 721, longitude: 1440)
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 8kB 1940-01-01T06:00:00 ... 2025-...
  * latitude    (latitude) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude   (longitude) float64 12kB -180.0 -179.8 -179.5 ... 179.5 179.8
    number      int64 8B 0
    expver      (valid_time) <U4 16kB '0001' '0001' '0001' ... '0005' '0005'
Data variables:
    tp          (valid_time, latitude, longitude) float32 4GB 0.0004425 ... 0...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-09-15T00:05 GRIB to CDM+CF via cfgrib-0.9.1...

## 1) Time Range: 

The first issue we should tackle is the time range. As you can see below, The ERA5 dataset spans over a longer period of time compared to GPCP. To fix this, we need to find a common time range and then slice both datasets over that common time range. Since GPCP doesn't span as far back as ERA5, we will choose the first time value for GPCP as our starting point. Between the two datasets, July 2025 seems to be a good endpoint. After we slice and define our new dataset, we can free up some RAM by deleting the orginial GPCP and ERA5 datasets as we will no longer be using them. 

In [17]:
# We print the start and end dates for both of our datasets: 
# We want to see what common range should be used for our analysis.
print('GPCP Date Range:')
print(GPCP['time'].min().values)
print(GPCP['time'].max().values)

print('\nERA5 Date Range:')
print(ERA5['valid_time'].min().values)
print(ERA5['valid_time'].max().values)

GPCP Date Range:
1979-01-01T00:00:00.000000000
2025-07-01T00:00:00.000000000

ERA5 Date Range:
1940-01-01T06:00:00.000000000
2025-08-01T06:00:00.000000000


In [18]:
# We slice and define the GPCP Dataset within the common date range:
gpcp = GPCP.sel(time=slice(GPCP['time'].min(), '2025-07'))
gpcp

<xarray.Dataset> Size: 23MB
Dimensions:    (time: 559, nv: 2, lat: 72, lon: 144)
Coordinates:
  * time       (time) datetime64[ns] 4kB 1979-01-01 1979-02-01 ... 2025-07-01
  * lat        (lat) float32 288B -88.75 -86.25 -83.75 ... 83.75 86.25 88.75
  * lon        (lon) float32 576B 1.25 3.75 6.25 8.75 ... 353.8 356.2 358.8
Dimensions without coordinates: nv
Data variables:
    time_bnds  (time, nv) datetime64[ns] 9kB 1979-01-01 ... 2025-08-01
    lat_bnds   (lat, nv) float32 576B -90.0 -87.5 -87.5 -85.0 ... 87.5 87.5 90.0
    lon_bnds   (lon, nv) float32 1kB 0.0 2.5 2.5 5.0 ... 355.0 357.5 357.5 360.0
    precip     (time, lat, lon) float32 23MB 0.0 0.0 0.0 ... 1.98 1.976 1.978
Attributes: (12/18)
    Conventions:           CF-1.0
    curator:               Dr. Jian-Jian Wang\nESSIC, University of Maryland ...
    citation:              Adler, R.F., G.J. Huffman, A. Chang, R. Ferraro, P...
    title:                 GPCP Version 2.3 Combined Precipitation Dataset (F...
    platform:              NOAA POES (Polar Orbiting Environmental Satellites)
    source_obs:            CDR RSS SSMI/SSMIS Tbs over ocean \nCDR SSMI/SSMIS...
    ...                    ...
    source:                https://www.ncei.noaa.gov/data/global-precipitatio...
    source_documentation:  https://www.ncdc.noaa.gov/cdr/atmospheric/precipit...
    NCO:                   4.6.9
    history:               Generated at NOAA/ESRL PSD Sep 9 2016 based on dat...
    References:            http://www.psl.noaa.gov/data/gridded/data.gpcp.html
    data_comment:          Interim data covers 2025/06 through latest.

In [19]:
# We also slice and define the ERA5 Dataset in the common date range:
era5 = ERA5.sel(valid_time=slice(GPCP['time'].min(), '2025-07'))
era5

<xarray.Dataset> Size: 2GB
Dimensions:     (valid_time: 559, latitude: 721, longitude: 1440)
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 4kB 1979-01-01T06:00:00 ... 2025-...
  * latitude    (latitude) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude   (longitude) float64 12kB -180.0 -179.8 -179.5 ... 179.5 179.8
    number      int64 8B 0
    expver      (valid_time) <U4 9kB '0001' '0001' '0001' ... '0001' '0005'
Data variables:
    tp          (valid_time, latitude, longitude) float32 2GB 0.0002937 ... 0...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-09-15T00:05 GRIB to CDM+CF via cfgrib-0.9.1...

## 2) Interpolation:

Now that our datasets have a common range, we need to ensure that the grid point structure is the same between the two. Esspecially since GPCP is more coarse than ERA5. Otherwise, there is no clear way to compare grid points between the two datasets. To keep things simple, we will reduce the resolution of the era5 to match the gpcp dataset. 

##### NOTE: You need to ensure that the coordinates and variable names match between the two datasets, otherwise interp_like() may not work properly. 

Once the interpolation is done, we can save both the gpcp and the era5 interpolated datasets as netCDF files which can be loaded and used by our main project file. That way we can load these files without having to repeat the procedure in this notebook.

In [20]:
# We free up memory in our computer by deleting the original dataset imports:
# We no longer need these as we will use gpcp and era5 instead.
del GPCP
del ERA5

In [21]:
# We rename the era5 variable and coordinate names to match those in gpcp: 
# This is required to use the .interp_like() command.
era5 = era5.rename({'tp':'precip'})
era5 = era5.rename({'valid_time':'time'})
era5 = era5.rename({'latitude':'lat'})
era5 = era5.rename({'longitude':'lon'})

era5

<xarray.Dataset> Size: 2GB
Dimensions:  (time: 559, lat: 721, lon: 1440)
Coordinates:
  * time     (time) datetime64[ns] 4kB 1979-01-01T06:00:00 ... 2025-07-01T06:...
  * lat      (lat) float64 6kB 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * lon      (lon) float64 12kB -180.0 -179.8 -179.5 ... 179.2 179.5 179.8
    number   int64 8B 0
    expver   (time) <U4 9kB '0001' '0001' '0001' '0001' ... '0001' '0001' '0005'
Data variables:
    precip   (time, lat, lon) float32 2GB 0.0002937 0.0002937 ... 0.000248
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-09-15T00:05 GRIB to CDM+CF via cfgrib-0.9.1...

In [22]:
era5_interp = era5.interp_like(gpcp,
                               method='nearest')

era5_interp

<xarray.Dataset> Size: 23MB
Dimensions:  (time: 559, lat: 72, lon: 144)
Coordinates:
  * time     (time) datetime64[ns] 4kB 1979-01-01 1979-02-01 ... 2025-07-01
  * lat      (lat) float32 288B -88.75 -86.25 -83.75 ... 83.75 86.25 88.75
  * lon      (lon) float32 576B 1.25 3.75 6.25 8.75 ... 351.2 353.8 356.2 358.8
    number   int64 8B 0
    expver   (time) <U4 9kB '0001' '0001' '0001' '0001' ... '0001' '0001' '0005'
Data variables:
    precip   (time, lat, lon) float32 23MB nan nan nan nan ... nan nan nan nan
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-09-15T00:05 GRIB to CDM+CF via cfgrib-0.9.1...

In [11]:
# We save our datasets in the netCDF file format:
gpcp.to_netcdf('gpcp.nc')
era5_interp.to_netcdf('era5.nc')